In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df_train = pd.read_csv('data/train.csv')
df_test = pd.read_csv('data/test.csv')

In [3]:
df_train.head()

,id,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f110,f111,f112,f113,f114,f115,f116,f117,f118,claim
0,0,0.10859,0.004314,-37.566,0.017364,0.28915,-10.25100,135.12,168900.0,3.992400e+14,...,-12.2280,1.7482,1.90960,-7.11570,4378.80,1.2096,8.613400e+14,140.1,1.01770,1
1,1,0.10090,0.299610,11822.000,0.276500,0.45970,-0.83733,1721.90,119810.0,3.874100e+15,...,-56.7580,4.1684,0.34808,4.14200,913.23,1.2464,7.575100e+15,1861.0,0.28359,0
2,2,0.17803,-0.006980,907.270,0.272140,0.45948,0.17327,2298.00,360650.0,1.224500e+13,...,-5.7688,1.2042,0.26290,8.13120,45119.00,1.1764,3.218100e+14,3838.2,0.40690,1
3,3,0.15236,0.007259,780.100,0.025179,0.51947,7.49140,112.51,259490.0,7.781400e+13,...,-34.8580,2.0694,0.79631,-16.33600,4952.40,1.1784,4.533000e+12,4889.1,0.51486,1
4,4,0.11623,0.502900,-109.150,0.297910,0.34490,-0.40932,2538.90,65332.0,1.907200e+15,...,-13.6410,1.5298,1.14640,-0.43124,3856.50,1.4830,-8.991300e+12,NaN,0.23049,1


In [4]:
data = df_train.sample(frac=0.9, random_state=786)
data_unseen = df_train.drop(data.index)

data.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Modeling: (862127, 120)
Unseen Data For Predictions: (95792, 120)


In [5]:
from pycaret.classification import *
exp_mclf101 = setup(data = data, target = 'claim', session_id=123, use_gpu = True) 

,Description,Value
0,session_id,123
1,Target,claim
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(862127, 120)"
5,Missing Values,True
6,Numeric Features,119
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [7]:
xgboost = create_model('xgboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7161,0.7761,0.6476,0.7486,0.6945,0.4318,0.4358
1,0.7177,0.7764,0.6538,0.7479,0.6977,0.4351,0.4386
2,0.7145,0.7712,0.6498,0.7447,0.6940,0.4287,0.4322
3,0.7189,0.7770,0.6557,0.7489,0.6992,0.4375,0.4409
4,0.7163,0.7751,0.6491,0.7483,0.6952,0.4324,0.4363
5,0.7162,0.7757,0.6504,0.7473,0.6954,0.4320,0.4357
6,0.7158,0.7721,0.6530,0.7451,0.6961,0.4314,0.4347
7,0.7172,0.7770,0.6530,0.7476,0.6971,0.4342,0.4377
8,0.7139,0.7735,0.6464,0.7456,0.6925,0.4276,0.4314


In [6]:
#best = compare_models()

In [9]:
tuned_best = tune_model(xgboost)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.4992,0.7580,0.9996,0.4987,0.6655,0.0017,0.0239
1,0.4993,0.7578,0.9996,0.4988,0.6655,0.0020,0.0269
2,0.4991,0.7532,0.9995,0.4987,0.6654,0.0015,0.0210
3,0.4992,0.7552,0.9997,0.4988,0.6655,0.0017,0.0259
4,0.4989,0.7560,0.9996,0.4986,0.6654,0.0012,0.0199
5,0.4991,0.7562,0.9995,0.4987,0.6654,0.0015,0.0217
6,0.4991,0.7555,0.9997,0.4987,0.6655,0.0016,0.0240
7,0.4992,0.7600,0.9995,0.4988,0.6655,0.0018,0.0239
8,0.4991,0.7564,0.9994,0.4987,0.6654,0.0016,0.0213


In [11]:
#make predictions
new_prediction = predict_model(tuned_best, data=data_unseen)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Extreme Gradient Boosting,0.4977,0.7598,0.9997,0.4973,0.6642,0.0013,0.0209


In [17]:
#fill missing values with mean
df_train.fillna(df_train.mean(), inplace=True)
df_test.fillna(df_train.mean(), inplace=True)

In [18]:
X, y = df_train.iloc[:, 1:-1].values, df_train.iloc[:, -1].values
X_test = df_test.iloc[:, 1:].values

In [19]:
#split data into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=0)

In [20]:
X_train.shape

(766335, 118)

In [21]:
#standardize data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)
X_test = sc.transform(X_test)

In [22]:
#build model
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [4]:
#optimize using optuna
from optuna import create_study
from optuna.samplers import TPESampler
from optuna.pruners import SuccessiveHalvingPruner

In [23]:
#optimize using optuna
def objective(trial):
    model = XGBClassifier(n_estimators= trial.suggest_categorical('n_estimators', [300, 500, 700, 900, 1000]),
                            max_depth= trial.suggest_int('max_depth', 1, 10),
                            learning_rate= trial.suggest_loguniform('learning_rate', 1e-5, 1e-1),
                            subsample= trial.suggest_uniform('subsample', 0.5, 1),
                            colsample_bytree= trial.suggest_uniform('colsample_bytree', 0.5, 1),
                            objective='binary:logistic',
                            n_jobs=-1,
                            random_state=0)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)
    return 1 - np.mean(y_pred == y_val)

In [27]:
#optimize using optuna
study = create_study(
    sampler=TPESampler(),
    pruner=SuccessiveHalvingPruner(
        min_resource=1,
        reduction_factor=5,
        min_early_stopping_rate=0.2
    ),
    load_if_exists=True
)

[I 2022-05-29 14:02:22,988] A new study created in memory with name: no-name-9972af08-5b4e-46ca-91ff-3b1462ca31c4


In [28]:
#optimize using optuna
study.optimize(objective, n_trials=100)

[I 2022-05-29 14:04:32,312] Trial 0 finished with value: 0.4061560464339402 and parameters: {'n_estimators': 900, 'max_depth': 2, 'learning_rate': 0.0036120676194116566, 'subsample': 0.7203214648826338, 'colsample_bytree': 0.7731878229851343}. Best is trial 0 with value: 0.4061560464339402.
[I 2022-05-29 14:06:58,851] Trial 1 finished with value: 0.3093629948221146 and parameters: {'n_estimators': 900, 'max_depth': 2, 'learning_rate': 0.03129604645643668, 'subsample': 0.9281344153791051, 'colsample_bytree': 0.9971261332705744}. Best is trial 1 with value: 0.3093629948221146.
[I 2022-05-29 14:11:04,295] Trial 2 finished with value: 0.324395565391682 and parameters: {'n_estimators': 900, 'max_depth': 4, 'learning_rate': 0.006796115469862539, 'subsample': 0.692791345575282, 'colsample_bytree': 0.8111306302936498}. Best is trial 1 with value: 0.3093629948221146.
[I 2022-05-29 14:12:21,483] Trial 3 finished with value: 0.4809169868047436 and parameters: {'n_estimators': 500, 'max_depth': 2,

KeyboardInterrupt: 

In [29]:
#best hyperparameters
study.best_params

{'n_estimators': 900,
 'max_depth': 2,
 'learning_rate': 0.03129604645643668,
 'subsample': 0.9281344153791051,
 'colsample_bytree': 0.9971261332705744}

In [30]:
xgb = XGBClassifier(
    n_estimators=study.best_params['n_estimators'],
    max_depth=study.best_params['max_depth'],
    learning_rate=study.best_params['learning_rate'],
    subsample=study.best_params['subsample'],
    colsample_bytree=study.best_params['colsample_bytree'],
    objective='binary:logistic',
    n_jobs=-1,
    gpus=0,
    random_state=0
)

In [31]:
xgb.fit(X_train, y_train)

[14:41:35] WARNING: ../src/learner.cc:627: 
Parameters: { "gpus" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.




XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.9971261332705744, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, gamma=0, gpu_id=-1,
              gpus=0, grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.03129604645643668,
              max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=2,
              max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=900, n_jobs=-1,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, ...)

In [32]:
xgb.score(X_val, y_val)

0.6906370051778854